In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
import torch.utils.data as Data
import torch.nn.functional as F
# from torchmetrics import R2Score
import os

d:\LenovoSoftstore\Anaconda\envs\yolov5\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
PATH='D:\\Deutschland\\FUB\\master_thesis\\data\\gee\\output'
DATA_DIR = os.path.join(PATH, 'monthly_mean')
LABEL_CSV = 'label.csv'

label_path = os.path.join(PATH, LABEL_CSV)
files = os.listdir(DATA_DIR)

Hyperparameters

In [4]:
BATCH_SIZE = 128
LR = 0.01
EPOCH = 500

input_size = 3
hidden_size = 64
num_layers = 1
num_classes = 10
layer1_dim = 400
layer2_dim = 128

Load csv file to ndarray

In [5]:
labels = pd.read_csv(label_path, sep=',', header=0, index_col=['id'])
x_list = []
y_list = []
for index, row in labels.iterrows():
    df_path = os.path.join(DATA_DIR, f'{index}.csv')
    df = pd.read_csv(df_path, sep=',', header=0, index_col=['date'])
    x = np.array(df, dtype=int)
    x = x.reshape(-1)
    y = row[0]
    x_list.append(x)
    y_list.append(y)

x_data = np.array(x_list)
y_data = np.array(y_list)

In [6]:
y_data.shape

(7935,)

Build train dataset

In [21]:
# embedding
embedding = nn.Embedding(8000, 3)
x_set = torch.from_numpy(x_data)
y_set = torch.from_numpy(y_data)
x_set = embedding(x_set).detach()

In [33]:
# split dataset
dataset = Data.TensorDataset(x_set, y_set)
size = len(dataset)
train_size, test_size = round(0.8 * size), round(0.2 * size)
train_dataset, test_dataset = Data.random_split(dataset, [train_size, test_size])
# data_loader
train_loader = Data.DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)
test_loader = Data.DataLoader(test_dataset,batch_size=test_size, shuffle=False,num_workers=2)

Neuron network

In [7]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc0 = nn.Linear(275, 512)
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 4)

    def forward(self, x:torch.Tensor):
        # x = x.view(x.size(0), -1)
        x = F.relu(self.fc0(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=1)
        return x

In [11]:
# LSTM model
class simpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, layer1_dim, layer2_dim, num_layers, num_classes):
        super(simpleLSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.linear1 = nn.Linear(hidden_size, layer1_dim, layer2_dim)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(layer1_dim, layer2_dim)
        self.relu2 = nn.ReLU()
        self.fc = nn.Linear(layer2_dim, num_classes)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        lstm_out, _ = self.lstm(x, (h0, c0))
        tmp1 = self.relu1(self.linear1(lstm_out[:,-1,:]))
        tmp2 = self.relu2(self.linear2(tmp1))
        out = self.fc(tmp2)
        return out

In [12]:
model = simpleLSTM(input_size, hidden_size, layer1_dim, layer2_dim, num_layers, num_classes).to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), LR)

Train and evaluate model

In [14]:
def train(model: nn.Module, epoch:int):
    model.train()
    total_loss = 0
    for batch_idx, (inputs, target) in enumerate(train_loader, 0):
        inputs = inputs.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        # forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print('[Epoch %d] loss: %.3f' % (epoch + 1, total_loss))

In [24]:
# train model
total_step = len(train_loader)
acc_list = []
model.train()
for epoch in range(EPOCH):
    for i, (input, label) in enumerate(train_loader):
        input = input.to(device)
        label = label.to(device)
        # forward pass
        output = model(input)
        loss = criterion(output, label)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 记录精度
        total = label.size(0)
        # torch.max(x,1) 按行取最大值
        # output每一行的最大值存在_中，每一行最大值的索引存在predicted中
        # output的每一行的每个元素的值表示是这一类的概率，取最大概率所对应的类作为分类结果
        # 也就是找到最大概率的索引
        _,predicted = torch.max(output.data,1)
        # .sum()计算出predicted和label相同的元素有多少个，返回的是一个张量，.item()得到这个张量的数值(int型)
        correct = (predicted == label).sum().item()
        acc_list.append(correct/total)

        if i % 50 == 0:
            print('Epoch[{}/{}],Step[{}/{}],Loss:{:.4f},Accuracy:{:.2f}%'
            .format(epoch+1,EPOCH,i+50,total_step,loss.item(),(correct/total)*100))

Epoch[1/500],Step[50/50],Loss:1.7979,Accuracy:28.12%
Epoch[2/500],Step[50/50],Loss:1.9120,Accuracy:24.22%
Epoch[3/500],Step[50/50],Loss:2.0677,Accuracy:23.44%
Epoch[4/500],Step[50/50],Loss:1.8586,Accuracy:25.78%
Epoch[5/500],Step[50/50],Loss:1.8451,Accuracy:35.94%
Epoch[6/500],Step[50/50],Loss:1.8673,Accuracy:21.88%
Epoch[7/500],Step[50/50],Loss:1.8265,Accuracy:32.03%
Epoch[8/500],Step[50/50],Loss:1.9940,Accuracy:24.22%
Epoch[9/500],Step[50/50],Loss:2.0662,Accuracy:25.00%
Epoch[10/500],Step[50/50],Loss:1.8601,Accuracy:30.47%
Epoch[11/500],Step[50/50],Loss:1.7805,Accuracy:28.91%
Epoch[12/500],Step[50/50],Loss:1.7475,Accuracy:37.50%
Epoch[13/500],Step[50/50],Loss:1.6781,Accuracy:32.81%
Epoch[14/500],Step[50/50],Loss:1.7473,Accuracy:32.81%
Epoch[15/500],Step[50/50],Loss:1.5951,Accuracy:35.94%
Epoch[16/500],Step[50/50],Loss:1.4539,Accuracy:40.62%
Epoch[17/500],Step[50/50],Loss:1.4031,Accuracy:53.12%
Epoch[18/500],Step[50/50],Loss:1.4018,Accuracy:49.22%
Epoch[19/500],Step[50/50],Loss:1.1626

KeyboardInterrupt: 

In [36]:
# validate the model
# model = torch.load('\model.pkl')
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for (value, label) in test_loader:
        value = value.to(device)
        label = label.to(device)
        outputs = model(value)
        total += label.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == label).sum().item()
    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

Test Accuracy of the model: 78.89098928796471 %


In [10]:
def test(model: nn.Module):
    model.eval()
    r2 = 0.
    with torch.no_grad():
        for (values, labels) in test_loader:
            values = values.to(device)
            labels = labels.to(device)
            outputs = model(values)
            num = labels.size(0)
            # transpose matrics
            outputs = outputs.t()
            labels = labels.t()
            r2score = R2Score(num_outputs=num, multioutput='uniform_average').to(device)
            r2 = r2score(labels, outputs).item()
    print('R^2 on test set: %.2f' % r2)